In [2]:
!pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/ac/64/8d0c8937dee86c286625bddb1902efacc3e22f2b619f5b5a8df29fe5217b/matplotlib-3.10.5-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/32/5c/1ee32d1c7956923202f00cf8d2a14a62ed7517bdc0ee1e55301227fc273c/contourpy-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/05/57/767e31e48861045d89691128bd81fd4c62b62150f9a17a666f731ce4f197/fonttools-4.59.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install seaborn

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.3 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# tqdm을 pandas에 적용하기 위한 설정
tqdm.pandas()

print("✅ 라이브러리 임포트 완료.")

✅ 라이브러리 임포트 완료.


In [9]:
# 분석할 모델의 토크나이저 (우리 최종 모델과 동일해야 함)
MODEL_NAME = "google/flan-t5-large" 
# 분석할 훈련 데이터 파일
TRAIN_CSV_PATH = "../data/processed/train.csv"

print(f"분석 대상 파일: {TRAIN_CSV_PATH}")
print(f"사용할 토크나이저: {MODEL_NAME}")

분석 대상 파일: ../data/processed/train.csv
사용할 토크나이저: google/flan-t5-large


In [10]:
try:
    print("토크나이저를 로딩합니다...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    print(f"'{TRAIN_CSV_PATH}' 파일을 로딩합니다...")
    df = pd.read_csv(TRAIN_CSV_PATH)
    
    print("\n✅ 데이터 로드 완료!")
    print(f"총 샘플 수: {len(df)}개")
    display(df.head())
    
except FileNotFoundError:
    print(f"🔥🔥🔥 에러: '{TRAIN_CSV_PATH}' 파일을 찾을 수 없습니다.")

토크나이저를 로딩합니다...
'../data/processed/train.csv' 파일을 로딩합니다...

✅ 데이터 로드 완료!
총 샘플 수: 11832개


,input_text,english_summary
0,summarize: topic: Room Service Requests dialog...,Mr. Smith contacts the room service and places...
1,summarize: topic: Business Issues dialogue: #P...,It appears that #Person2#'s business is not pe...
2,summarize: topic: Everyday Conversations dialo...,#Person1# and #Person2# engage in a conversati...
3,summarize: topic: 경기 Schedule dialogue: #Perso...,#Person2# informed #Person1# that the match wa...
4,summarize: topic: Reflections on Life dialogue...,#Person1# and #Person2# are engaged in a conve...


In [11]:
print("모든 요약문의 토큰 길이를 계산합니다... (샘플 수에 따라 몇 분 정도 걸릴 수 있습니다)")

# 'english_summary' 컬럼의 각 행에 대해 토큰 길이를 계산하여 새 컬럼에 저장
df['summary_token_length'] = df['english_summary'].progress_apply(
    lambda x: len(tokenizer.encode(str(x)))
)

print("\n✅ 토큰 길이 계산 완료!")
display(df[['english_summary', 'summary_token_length']].head())

모든 요약문의 토큰 길이를 계산합니다... (샘플 수에 따라 몇 분 정도 걸릴 수 있습니다)


  0%|          | 0/11832 [00:00<?, ?it/s]


✅ 토큰 길이 계산 완료!


,english_summary,summary_token_length
0,Mr. Smith contacts the room service and places...,21
1,It appears that #Person2#'s business is not pe...,111
2,#Person1# and #Person2# engage in a conversati...,113
3,#Person2# informed #Person1# that the match wa...,47
4,#Person1# and #Person2# are engaged in a conve...,34


In [12]:
# 계산된 길이 데이터
summary_lengths = df['summary_token_length']

# 통계 계산
max_len = summary_lengths.max()
mean_len = summary_lengths.mean()
median_len = summary_lengths.median()
percentile_95 = summary_lengths.quantile(0.95)
percentile_99 = summary_lengths.quantile(0.99)
percentile_999 = summary_lengths.quantile(0.999)
recommended_len = int(np.ceil(percentile_99))

# 결과 출력
print("--- 📊 요약문 길이 분석 결과 📊 ---")
print(f"평균 길이 (Mean):     {mean_len:.2f} 토큰")
print(f"중간값 길이 (Median):   {median_len:.2f} 토큰")
print(f"상위 95% 지점 길이:     {percentile_95:.2f} 토큰")
print(f"상위 99% 지점 길이:     {percentile_99:.2f} 토큰")
print(f"상위 99.9% 지점 길이:   {percentile_999:.2f} 토큰")
print(f"최대 길이 (Max):        {max_len} 토큰")
print("------------------------------------")
print(f"\n✅ 추천 `max_length`: {recommended_len} (데이터의 99%를 손실 없이 커버하는 길이)")

--- 📊 요약문 길이 분석 결과 📊 ---
평균 길이 (Mean):     54.50 토큰
중간값 길이 (Median):   47.00 토큰
상위 95% 지점 길이:     109.00 토큰
상위 99% 지점 길이:     177.69 토큰
상위 99.9% 지점 길이:   386.85 토큰
최대 길이 (Max):        461 토큰
------------------------------------

✅ 추천 `max_length`: 178 (데이터의 99%를 손실 없이 커버하는 길이)
